In [51]:
import psycopg2
import pandas as pd
from datetime import datetime
import numpy as np

mapping = {
    "3OB05285": "12P52852",
    "3OB05287": "14P52874",
    "3OB05259": "15P52595",
    "3OB05283": "16P52836",
    "3OB05286": "17P52867",
    "3OB05294": "18P529422",
    "3OB05295": "19P529523",
    "3OB05255": "1P525501",
    "3OB05292": "20P529221",
    "3OB05303": "21P530324",
    "3OB05306": "22P530624",
    "3OB05305": "23P530525",
    "3OB05311": "24P531126",
    "3OB05314": "25P531426",
    "3OB05315": "26P531527",
    "3OB05322": "27P532248",
    "3OB05323": "28P532349",
    "3OB05324": "29P532451",
    "3OB05258": "2P525802",
    "3OB05326": "30P532652",
    "3OB05327": "31P532753",
    "3OB05345": "32P534563",
    "3OB05346": "33P534612",
    "3OB05347": "34P534728",
    "3OB05360": "35P536046",
    "3OB05361": "36P536158",
    "3OB05362": "37P536299",
    "3OB05363": "38P536381",
    "3OB05364": "39P536458",
    "3OB05284": "3P528403",
    "3OB05365": "40P536571",
    "3OB05366": "41P536622",
    "3OB05367": "42P536787",
    "3OB05256": "4P525604",
    "3OB05257": "5P525705",
    "3OB05288": "6P528806",
    "3OB05260": "7P526007",
    "3OB05289": "8P528918",
    "3OB05261": "9P526109",
    "3OB05254": "SIPM5254"
}
reversed_mapping = {v: k for k, v in mapping.items()}

def convert_contract(contract):
    parts = contract.split()
    if len(parts) < 3:
        return contract  # or whatever you want to do in this case
    else:
        # Check if there are two decimal places
        decimal_parts = parts[2].split(".")
        if len(decimal_parts) > 1 and len(decimal_parts[1]) == 2:
            # Add an extra '0' if there are two decimal places
            parts[2] = parts[2] + '0'
        return ''.join(parts[:2]) + parts[2]

def fetch_large_orders(date):
    connection = psycopg2.connect(host="10.7.8.59", database="fixtransactions", user="scalp", password="QAtr@de442", port='5433')
    cursor = connection.cursor()
    query = f"""
        SELECT tag1 as account, tag167 as order_type, tag38 as quantity, tag11 as order_id, tag52 as date, tag55 as symbol, 
        tag200 as maturityMY, tag201 as put_call, tag202 as strike_price, tag205 as maturityD, tag541 as full_exp_date, tag54 as side,
        tag39 as status, tag44 as price, tag49 as broker, tag442 as spread_or_single, tag40 as orderType
        FROM fixmsg
        WHERE tag52 >= TIMESTAMP '{date}'
        AND tag52 < TIMESTAMP '{date}' + INTERVAL '1 DAY'
        AND tag38 >= '100'
        AND tag39 = '0'
        """
    cursor.execute(query)
    results = cursor.fetchall()
    cursor.close()
    connection.close()
    
    columns = ['Account', 'order_type', 'quantity', 'order_id', 'date', 'symbol', 'maturityMY', 'put_call', 'strike_price', 'maturityD', 'full_exp_date', 'side', 'status', 'price', 'broker', 'spread_or_single', 'orderType']
    large_orders_df = pd.DataFrame(results, columns=columns)
    return large_orders_df

def calculate_positions(start_time, end_date, account_num, contract_symbol):
    # Establish a connection to the PostgreSQL database
    connection = psycopg2.connect(host="10.7.8.59", database="fixtransactions", user="scalp", password="QAtr@de442", port='5432')
    cursor = connection.cursor()
    query = f"""
        SELECT SUM(quantity)
        FROM trades
        WHERE trading_date = timestamp '{start_time}'
        AND trade_timestamp <= timestamp '{end_date}'
        AND clearing_account = '{account_num}'
        AND symbol = '{contract_symbol}'
        """
    cursor.execute(query)
    position_at_time = cursor.fetchone()[0]
    cursor.close()
    connection.close()

    return position_at_time

def get_large_orders_df(date):
    start_time = date
    large_orders_df = fetch_large_orders(date)
    large_orders_df["Account"] = large_orders_df["Account"].replace(reversed_mapping)
    large_orders_df["date"] = large_orders_df["date"].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3] + '-04')
    large_orders_df['Option/Equity'] = large_orders_df['symbol'].apply(lambda x: 'Option' if x[-1] == '0' else 'Equity')
    large_orders_df["Side"] = large_orders_df["quantity"].apply(lambda x: "Buy" if x > 0 else "Sell")
    large_orders_df['Contract_Name'] = large_orders_df.apply(compute_contract_name, axis=1)
    large_orders_df['option_equity'] = large_orders_df['strike_price'].apply(lambda x: "Equity" if pd.isnull(x) else "Option")
    large_orders_df.drop(columns=["maturityMY", "maturityD", "full_exp_date"], inplace=True)
    large_orders_df['put_call'] = large_orders_df['put_call'].replace({0: 'put', 1: 'call'})
    large_orders_df['side'] = large_orders_df['side'].replace({"1": 'Buy', "2": 'Sell', "3": 'Buy minus', "4": 'Sell plus', "5": 'Sell short'})
    large_orders_df['status'] = large_orders_df['status'].replace({"0": "New Order", "1": 'Partially filled', "2": 'Filled'})
    large_orders_df['spread_or_single'] = large_orders_df['spread_or_single'].fillna(0).astype(int).replace({1: "single", 2: "spread", 3: "spread", 0: "single"})
    large_orders_df['orderType'] = large_orders_df['orderType'].astype(int).replace({1: "Market", 2: "Limit", 3: "Stop", 4: "Stop Limit", 5: "Market on Close"})
    large_orders_df["Contract_Name"] = large_orders_df["Contract_Name"].fillna(large_orders_df["symbol"])
    print(f"this is lodf {large_orders_df}")
    positions = []

    for index, row in large_orders_df.iterrows():
        account_num = str(row['Account'])
        contract_symbol = str(row['Contract_Name'])
        end_dates = row['date']
        position = calculate_positions(start_time, end_dates, account_num, contract_symbol)
        positions.append(position)

    large_orders_df['Position'] = positions
    large_orders_df["Position"] = large_orders_df["Position"].apply(convert_value)


    return large_orders_df

def compute_contract_name(row):
    if row['put_call'] == 0:
        option_type = 'P'
    elif row['put_call'] == 1:
        option_type = 'C'
    else:
        return None

    try:
        if row['full_exp_date'] is not None:
            expiration_date = row['full_exp_date']
        else:
            expiration_date = datetime.strptime(f"{int(row['maturityMY'])}{int(row['maturityD'])}", '%Y%m%d').strftime('%Y%m%d')
    except ValueError:
        return None

    strike_price = "{:.3f}".format(float(row['strike_price']))
    option_contract_name = f"{row['symbol']}{expiration_date}{option_type}{strike_price}"
    return option_contract_name

def convert_value(value):
    if value is None:
        return value
    try:
        # Try to convert the value to float first
        val = float(value)
        # If the value is an integer number, convert it to int
        if val.is_integer():
            val = int(val)
        return str(val)
    except ValueError:
        # If it can't be converted to a float (it's not a number), return it as is
        return value


# sod_pos = pd.read_csv("/home/elliott/Development/files/Scalp_Trade_EXT011_20230602.csv")
column_names = ['Type', 'Unknown1', 'Unknown2', 'Unknown3', 'Account', 'Unknown4', 'Currency', 'Contract_Name', 'Unknown5', 'Price', 'Initial_Position']
sod_pos = pd.read_csv("/home/elliott/Development/files/Scalp_Trade_EXT011_20230605.csv", sep=';', header=None, names=column_names)
sod_pos.drop(columns=["Type", "Unknown1", "Unknown2", "Unknown3", "Unknown4", "Currency", "Unknown5", "Price"], inplace=True)
sod_pos['Contract_Name'] = sod_pos['Contract_Name'].apply(convert_contract)

sod_pos.to_csv("starting_postitions_5th.csv")   
date = '2023-06-05'
large_orders_df = get_large_orders_df(date)

print("large_orders_df:")
print(large_orders_df[['Account', 'Contract_Name']])
print("\n")
print("sod_pos:")
print(sod_pos)

# Merge large_orders_df and sod_pos
merged_df = large_orders_df.merge(sod_pos, on=['Account', 'Contract_Name'], how='left')

# Identify and print matching rows
matching_rows = merged_df[~merged_df['Initial_Position'].isna()]
print("Matching Rows:")
print(matching_rows)

# Calculate the 'Combined Position' as you did before
merged_df['Combined Position'] = np.where(
    pd.isnull(merged_df['Initial_Position']),
    merged_df['Position'],
    merged_df['Position'].astype(float) + merged_df['Initial_Position'].astype(float)
)

# Continue as before
merged_df.to_csv(f"large_orders_df_{date}.csv")
merged_df



this is lodf       Account order_type quantity         order_id  \
0    3OB05284       None      100  320040202511861   
1    3OB05284       None      100  320040202511876   
2    3OB05284       None      100  320040202511888   
3    3OB05284       None      100  320040202511927   
4    3OB05284       None      100  320040202511983   
..        ...        ...      ...              ...   
259  3OB05260        OPT      100  320040202705744   
260  3OB05260        OPT      197  320040202712369   
261  3OB05260        OPT      197  320040202712420   
262  3OB05260        OPT      197  320040202713445   
263  3OB05260        OPT      197  320040202713490   

                           date symbol put_call strike_price        side  \
0    2023-06-05 13:48:14.883-04    XOP      NaN         None        Sell   
1    2023-06-05 13:48:16.142-04    XOP      NaN         None  Sell short   
2    2023-06-05 13:48:17.527-04    XOP      NaN         None        Sell   
3    2023-06-05 13:48:19.850-04   

,Account,order_type,quantity,order_id,date,symbol,put_call,strike_price,side,status,...,broker,spread_or_single,orderType,Option/Equity,Side,Contract_Name,option_equity,Position,Initial_Position,Combined Position
0,3OB05284,None,100,320040202511861,2023-06-05 13:48:14.883-04,XOP,NaN,None,Sell,New Order,...,DASHNORM,single,Limit,Equity,Buy,XOP,Equity,100,NaN,100
1,3OB05284,None,100,320040202511876,2023-06-05 13:48:16.142-04,XOP,NaN,None,Sell short,New Order,...,DASHNORM,single,Limit,Equity,Buy,XOP,Equity,100,NaN,100
2,3OB05284,None,100,320040202511888,2023-06-05 13:48:17.527-04,XOP,NaN,None,Sell,New Order,...,DASHNORM,single,Limit,Equity,Buy,XOP,Equity,100,NaN,100
3,3OB05284,None,100,320040202511927,2023-06-05 13:48:19.850-04,XOP,NaN,None,Sell,New Order,...,DASHNORM,single,Limit,Equity,Buy,XOP,Equity,100,NaN,100
4,3OB05284,None,100,320040202511983,2023-06-05 13:48:21.762-04,XOP,NaN,None,Sell,New Order,...,DASHNORM,single,Limit,Equity,Buy,XOP,Equity,100,NaN,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,3OB05260,OPT,100,320040202705744,2023-06-05 19:37:11.041-04,BITO,call,15,Sell,New Order,...,VOLANT,single,Limit,Equity,Buy,BITO20240119C15.000,Option,191,191.0,382.0
260,3OB05260,OPT,197,320040202712369,2023-06-05 19:49:06.465-04,BITO,call,15,Sell,New Order,...,VOLANT,single,Limit,Equity,Buy,BITO20240119C15.000,Option,191,191.0,382.0
261,3OB05260,OPT,197,320040202712420,2023-06-05 19:49:12.322-04,BITO,call,15,Sell,New Order,...,VOLANT,single,Limit,Equity,Buy,BITO20240119C15.000,Option,191,191.0,382.0
262,3OB05260,OPT,197,320040202713445,2023-06-05 19:51:31.455-04,BITO,call,15,Sell,New Order,...,VOLANT,single,Limit,Equity,Buy,BITO20240119C15.000,Option,191,191.0,382.0


In [2]:
import psycopg2
import pandas as pd
import numpy as np
from datetime import datetime

mapping = {
    "3OB05285": "12P52852",
    "3OB05287": "14P52874",
    "3OB05259": "15P52595",
    "3OB05283": "16P52836",
    "3OB05286": "17P52867",
    "3OB05294": "18P529422",
    "3OB05295": "19P529523",
    "3OB05255": "1P525501",
    "3OB05292": "20P529221",
    "3OB05303": "21P530324",
    "3OB05306": "22P530624",
    "3OB05305": "23P530525",
    "3OB05311": "24P531126",
    "3OB05314": "25P531426",
    "3OB05315": "26P531527",
    "3OB05322": "27P532248",
    "3OB05323": "28P532349",
    "3OB05324": "29P532451",
    "3OB05258": "2P525802",
    "3OB05326": "30P532652",
    "3OB05327": "31P532753",
    "3OB05345": "32P534563",
    "3OB05346": "33P534612",
    "3OB05347": "34P534728",
    "3OB05360": "35P536046",
    "3OB05361": "36P536158",
    "3OB05362": "37P536299",
    "3OB05363": "38P536381",
    "3OB05364": "39P536458",
    "3OB05284": "3P528403",
    "3OB05365": "40P536571",
    "3OB05366": "41P536622",
    "3OB05367": "42P536787",
    "3OB05256": "4P525604",
    "3OB05257": "5P525705",
    "3OB05288": "6P528806",
    "3OB05260": "7P526007",
    "3OB05289": "8P528918",
    "3OB05261": "9P526109",
    "3OB05254": "SIPM5254"
}

reversed_mapping = {v: k for k, v in mapping.items()}

def convert_contract(contract):
    parts = contract.split()
    if len(parts) < 3:
        return contract
    else:
        decimal_parts = parts[2].split(".")
        if len(decimal_parts) > 1 and len(decimal_parts[1]) == 2:
            parts[2] = parts[2] + '0'
        return ''.join(parts[:2]) + parts[2]

def fetch_large_orders(date):
    connection = psycopg2.connect(host="10.7.8.59", database="fixtransactions", user="scalp", password="QAtr@de442", port='5433')
    cursor = connection.cursor()
    query = f"""
        SELECT tag1 as account, tag167 as order_type, tag38 as quantity, tag11 as order_id, tag52 as date, tag55 as symbol, 
        tag200 as maturityMY, tag201 as put_call, tag202 as strike_price, tag205 as maturityD, tag541 as full_exp_date, tag54 as side,
        tag39 as status, tag44 as price, tag49 as broker, tag442 as spread_or_single, tag40 as orderType
        FROM fixmsg
        WHERE tag52 >= TIMESTAMP '{date}'
        AND tag52 < TIMESTAMP '{date}' + INTERVAL '1 DAY'
        AND tag38 >= '150'
        AND tag39 = '0'
        """
    cursor.execute(query)
    results = cursor.fetchall()
    cursor.close()
    connection.close()
    
    columns = ['Account', 'order_type', 'quantity', 'order_id', 'date', 'symbol', 'maturityMY', 'put_call', 'strike_price', 'maturityD', 'full_exp_date', 'side', 'status', 'price', 'broker', 'spread_or_single', 'orderType']
    large_orders_df = pd.DataFrame(results, columns=columns)
    return large_orders_df

def calculate_positions(start_time, end_date, account_num, contract_symbol):
    connection = psycopg2.connect(host="10.7.8.59", database="fixtransactions", user="scalp", password="QAtr@de442", port='5432')
    cursor = connection.cursor()
    query = f"""
        SELECT SUM(quantity)
        FROM trades
        WHERE trading_date = timestamp '{start_time}'
        AND trade_timestamp <= timestamp '{end_date}'
        AND clearing_account = '{account_num}'
        AND symbol = '{contract_symbol}'
        """
    cursor.execute(query)
    position_at_time = cursor.fetchone()[0]
    cursor.close()
    connection.close()

    return position_at_time

def get_large_orders_df(date):
    start_time = date
    large_orders_df = fetch_large_orders(date)
    large_orders_df["Account"] = large_orders_df["Account"].replace(reversed_mapping)
    large_orders_df["date"] = large_orders_df["date"].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3] + '-04')
    large_orders_df['Option/Equity'] = large_orders_df['symbol'].apply(lambda x: 'Option' if x[-1] == '0' else 'Equity')
    large_orders_df["Side"] = large_orders_df["quantity"].apply(lambda x: "Buy" if x > 0 else "Sell")
    large_orders_df['Contract_Name'] = large_orders_df.apply(compute_contract_name, axis=1)
    large_orders_df['option_equity'] = large_orders_df['strike_price'].apply(lambda x: "Equity" if pd.isnull(x) else "Option")
    large_orders_df.drop(columns=["maturityMY", "maturityD", "full_exp_date", "order_type"], inplace=True)
    large_orders_df['put_call'] = large_orders_df['put_call'].replace({0: 'put', 1: 'call'})
    large_orders_df['side'] = large_orders_df['side'].replace({"1": 'Buy', "2": 'Sell', "3": 'Buy minus', "4": 'Sell plus', "5": 'Sell short'})
    large_orders_df['status'] = large_orders_df['status'].replace({"0": "New Order", "1": 'Partially filled', "2": 'Filled'})
    large_orders_df['spread_or_single'] = large_orders_df['spread_or_single'].fillna(0).astype(int).replace({1: "single", 2: "spread", 3: "spread", 0: "single"})
    large_orders_df['orderType'] = large_orders_df['orderType'].astype(int).replace({1: "Market", 2: "Limit", 3: "Stop", 4: "Stop Limit", 5: "Market on Close"})
    large_orders_df["Contract_Name"] = large_orders_df["Contract_Name"].fillna(large_orders_df["symbol"])
    positions = []

    for index, row in large_orders_df.iterrows():
        account_num = str(row['Account'])
        contract_symbol = str(row['Contract_Name'])
        end_dates = row['date']
        position = calculate_positions(start_time, end_dates, account_num, contract_symbol)
        positions.append(position)

    large_orders_df['Position'] = positions
    large_orders_df["Position"] = large_orders_df["Position"].apply(convert_value)

    return large_orders_df

def compute_contract_name(row):
    if row['put_call'] == 0:
        option_type = 'P'
    elif row['put_call'] == 1:
        option_type = 'C'
    else:
        return None

    try:
        if row['full_exp_date'] is not None:
            expiration_date = row['full_exp_date']
        else:
            expiration_date = datetime.strptime(f"{int(row['maturityMY'])}{int(row['maturityD'])}", '%Y%m%d').strftime('%Y%m%d')
    except ValueError:
        return None

    strike_price = "{:.3f}".format(float(row['strike_price']))
    option_contract_name = f"{row['symbol']}{expiration_date}{option_type}{strike_price}"
    return option_contract_name

def convert_value(value):
    if value is None:
        return value
    try:
        val = float(value)
        if val.is_integer():
            val = int(val)
        return str(val)
    except ValueError:
        return value

def run(date):
    replace_date = date.replace('-', '')
    # sod_pos = pd.read_csv("/home/elliott/Development/files/Scalp_Trade_EXT011_20230602.csv")
    column_names = ['Type', 'Unknown1', 'Unknown2', 'Unknown3', 'Account', 'Unknown4', 'Currency', 'Contract_Name', 'Unknown5', 'Price', 'Initial_Position']
    sod_pos = pd.read_csv(f"/home/elliott/Development/files/Scalp_Trade_EXT011_{replace_date}.csv", sep=';', header=None, names=column_names)
    sod_pos.drop(columns=["Type", "Unknown1", "Unknown2", "Unknown3", "Unknown4", "Currency", "Unknown5", "Price"], inplace=True)
    sod_pos['Contract_Name'] = sod_pos['Contract_Name'].apply(convert_contract)

    sod_pos.to_csv(f"starting_postitions{date}.csv")   
    large_orders_df = get_large_orders_df(date)

    # Merge large_orders_df and sod_pos
    merged_df = large_orders_df.merge(sod_pos, on=['Account', 'Contract_Name'], how='left')

    # Calculate the 'Combined Position' as you did before
    merged_df['Net Position'] = np.where(
        pd.isnull(merged_df['Initial_Position']),
        merged_df['Position'],
        merged_df['Position'].astype(float) + merged_df['Initial_Position'].astype(float)
    )
    # Continue as before
    merged_df.to_csv(f"large_orders_df_{date}.csv")

run('2023-06-20')
